In [2]:
import requests
import bs4

url = "http://kodokanjudoinstitute.org/en/waza/list/"
response = requests.get(url)
soup = bs4.BeautifulSoup(response.text, "html.parser")
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="no" http-equiv="imagetoolbar">
<meta content="Kodokan,Judo,Kano Jigoro" name="keywords"/>
<meta content="The official WEB site of The Kodokan Judo Institute, Tokyo, Japan." name="description"/>
<link href="/en/common/styles/style.css" media="all" rel="stylesheet"/>
<link href="/styles/page.css" media="all" rel="stylesheet"/>
<link href="/styles/waza/list.css" media="all" rel="stylesheet"/>
<script src="//ajax.googleapis.com/ajax/libs/jquery/1.11.1/jquery.min.js"></script>
<script src="/common/scripts/jquery.cookie.js"></script>
<script src="/common/scripts/script.js"></script>
<!--[if lt IE 9]>
<script src="http://html5shiv.googlecode.com/svn/trunk/html5.js"></script>
<![endif]-->
<script type="text/javascript">
$(function(){
$("body").css("font-size",$.cookie('fsize'));
});
function font(size){
$("body").css("font-size",size);
$.cookie("fsize",size,{expires:7});
}
</script>
<title>Names of Judo Techniques 

In [18]:
import pandas as pd

# parse all the table tags from the page
tables = soup.find_all("table")
# print(tables[0])
# then for each table, extract the href (youtube video link), the technique name as part of that href, and the text description
res = []
for table in tables:
    for row in table.find_all("tr"):
        for cell in row.find_all("td"):
            for link in cell.find_all("a"):
                # get the youtube link
                youtube_link = link.get("href")
                # get the technique name from the link
                name = link.text.lower()

                # fill in the missing value for sasae (broken link)
                if "sasae-tsurikomi-ashi" in name:
                    youtube_link = "https://www.youtube.com/watch?v=699i--pvYmE"

                # get the text description
                description = cell.text.split("\n")[-1].strip()
                # print out the results
                res.append(
                    {
                        "youtube_link": youtube_link,
                        "name": name,
                        "description": description,
                    }
                )
df = pd.DataFrame(res)
df.head()

,youtube_link,name,description
0,https://youtu.be/zIq0xI0ogxk,seoi-nage,A technique to throw the opponent over the rig...
1,https://youtu.be/FQnOlCxo4oI,ippon-seoi-nage,A technique to throw the opponent over the rig...
2,https://youtu.be/vu1TMVNnq34,seoi-otoshi,A technique to pull down and throw the opponen...
3,https://youtu.be/4x6S3Q-Ktv8,tai-otoshi,A technique to pull down and throw the opponen...
4,https://youtu.be/cnHRhSy8yi4,kata-guruma,A technique to uplift the opponent on both sho...


In [19]:
# let's add this to our existing waza
waza = pd.read_csv("../data/datasets/waza.csv")
# drop the columns if they exist
if "youtube_link" in waza.columns:
    waza = waza.drop(columns=["youtube_link"])
if "description" in waza.columns:
    waza = waza.drop(columns=["description"])

# add the new columns
waza = waza.merge(df, on="name", how="left")
waza.to_csv("../data/datasets/waza.csv", index=False)
waza.to_json("../data/datasets/waza.json", orient="records")